In [1]:
pip install psycopg2 pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
def createDatabase():
    conn=psycopg2.connect("host=127.0.0.1 dbname=postgres port=5433 user=postgres password=NoBodyDoesItBetter@69")
    curr=conn.cursor()
    conn.set_session(autocommit=True)
    
    curr.execute("DROP DATABASE IF EXISTS taxis")
    curr.execute("CREATE DATABASE taxis")
    
    conn.close()
    
    conn=psycopg2.connect("host=127.0.0.1 dbname=taxis port=5433 user=postgres password=NoBodyDoesItBetter@69")
    curr=conn.cursor()
    
    return curr,conn

In [3]:
import psycopg2
import pandas as pd

In [4]:
customerId=pd.read_csv("./Taxi_Dataset/Customer_ID.csv")

In [5]:
customerId.head()

,Customer ID,Gender,Age,Income (USD/Month)
0,29290,Male,28,10813
1,27703,Male,27,9237
2,28712,Male,53,11242
3,28020,Male,23,23327
4,27182,Male,33,8536


In [6]:
taxiData=pd.read_csv("./Taxi_Dataset/Taxi_Data.csv")

In [7]:
taxiData.head()

,Transaction ID,Date of Travel,Company,City,Distance Travelled(KM),Price Charged,Cost of Trip
0,10000011,42377,Pink Cab,ATLANTA GA,30.45,370.95,313.635
1,10000012,42375,Pink Cab,ATLANTA GA,28.62,358.52,334.854
2,10000013,42371,Pink Cab,ATLANTA GA,9.04,125.20,97.632
3,10000014,42376,Pink Cab,ATLANTA GA,33.17,377.40,351.602
4,10000015,42372,Pink Cab,ATLANTA GA,8.73,114.62,97.776


In [8]:
transactionId=pd.read_csv("./Taxi_Dataset/Transaction_ID.csv")

In [9]:
transactionId.head()

,Transaction ID,Customer ID,Payment_Mode
0,10000011,29290,Card
1,10000012,27703,Card
2,10000013,28712,Cash
3,10000014,28020,Cash
4,10000015,27182,Card


In [10]:
customerId.dtypes

Customer ID            int64
Gender                object
Age                    int64
Income (USD/Month)     int64
dtype: object

In [11]:
taxiData.dtypes

Transaction ID              int64
Date of Travel              int64
Company                    object
City                       object
Distance Travelled(KM)    float64
Price Charged             float64
Cost of Trip              float64
dtype: object

In [12]:
transactionId.dtypes

Transaction ID     int64
Customer ID        int64
Payment_Mode      object
dtype: object

In [13]:
curr,conn=createDatabase()

In [14]:
customerId_table_create=("""CREATE TABLE IF NOT EXISTS customerId(
customer_id int PRIMARY KEY,
gender varchar,
age int,
income int
)""")

In [15]:
curr.execute(customerId_table_create)
conn.commit()

In [16]:
taxiData_table_create=("""CREATE TABLE IF NOT EXISTS taxiData(
transactionId int PRIMARY KEY,
dateOfTravel int,
company varchar,
city varchar,
distanceTravelled float,
priceCharged float,
costOfTrip float
)""")
curr.execute(taxiData_table_create)
conn.commit()

In [17]:
transactionId_table_create=("""CREATE TABLE IF NOT EXISTS transactionId(
transactionId int PRIMARY KEY,
customer_id int REFERENCES customerId(customer_id),
mode_of_payment varchar
)""")
curr.execute(transactionId_table_create)
conn.commit()

In [18]:
customerId_table_insert=(""" INSERT INTO customerId(
customer_id,
gender,
age,
income
)
VALUES(%s,%s,%s,%s)
""")

In [19]:
for i,row in customerId.iterrows():
    curr.execute(customerId_table_insert,list(row))

In [20]:
conn.commit()

In [29]:
taxiData_table_insert=(""" INSERT INTO taxiData(
transactionId,
dateOfTravel,
company,
city,
distanceTravelled,
priceCharged,
costOfTrip
)
VALUES(%s,%s,%s,%s,%s,%s,%s)
""")

In [30]:
for i,row in taxiData.iterrows():
    curr.execute(taxiData_table_insert,list(row))
    conn.commit()

In [31]:
transactionId_table_insert=(""" INSERT INTO transactionId(
transactionId,
customer_id,
mode_of_payment
)
VALUES(%s,%s,%s)
""")

In [32]:
for i,row in transactionId.iterrows():
    curr.execute(transactionId_table_insert,list(row))
    conn.commit()